In [1]:
# Environment Verification and Setup Cell
import sys
import os
from pathlib import Path
import subprocess

print("=" * 80)
print("ENVIRONMENT VERIFICATION & AUTO-SETUP")
print("=" * 80)

# 1. Python executable and version
print(f"\n1. Python Executable:")
print(f"   {sys.executable}")
print(f"\n2. Python Version:")
print(f"   {sys.version}")

# 3. Virtual environment check
venv_path = os.environ.get('VIRTUAL_ENV', None)
if venv_path:
    print(f"\n3. Virtual Environment:")
    print(f"   ✅ ACTIVE: {venv_path}")
else:
    print(f"\n3. Virtual Environment:")
    print(f"   ⚠️  NOT DETECTED")
    print(f"   Note: You can still use this notebook, dependencies will be installed automatically.")

# 4. Verify backtester package is available
print(f"\n4. Backtester Package:")
try:
    import backtester
    print(f"   ✅ FOUND (installed): {backtester.__file__}")
except ImportError:
    # If not installed, try adding to path
    code_path = Path('..') / 'code'
    if code_path.exists():
        sys.path.insert(0, str(code_path.absolute()))
        try:
            import backtester
            print(f"   ✅ FOUND (via path): {code_path.absolute()}")
        except ImportError:
            print(f"   ❌ NOT FOUND - Check ../code directory")
    else:
        print(f"   ❌ NOT FOUND - Installation required")

# 5. Check and auto-install dependencies
print(f"\n5. Dependencies (auto-installing if missing):")
dependencies = {
    'pandas': 'pandas',
    'numpy': 'numpy',
    'scipy': 'scipy',
    'plotly': 'plotly',
    'doltpy': 'doltpy'
}

all_ok = True
for name, module in dependencies.items():
    try:
        __import__(module)
        print(f"   ✅ {name}")
    except ImportError:
        print(f"   ⚠️  {name} - INSTALLING...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", module, "-q"])
            __import__(module)
            print(f"   ✅ {name} - INSTALLED SUCCESSFULLY")
        except Exception as e:
            print(f"   ❌ {name} - INSTALLATION FAILED: {e}")
            all_ok = False

# 6. Database path check
print(f"\n6. Database Path:")
db_path = Path('/Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options')
if db_path.exists():
    print(f"   ✅ Found: {db_path}")
else:
    print(f"   ❌ Not found: {db_path}")
    all_ok = False

# Summary
print(f"\n" + "=" * 80)
if all_ok:
    print("✅ ENVIRONMENT READY - All checks passed!")
    print("You may proceed with running the notebook cells.")
else:
    print("⚠️  PARTIAL SETUP - Some dependencies may be missing")
    print("Try installing manually: pip install pandas numpy scipy plotly doltpy")
print("=" * 80)

ENVIRONMENT VERIFICATION & AUTO-SETUP

1. Python Executable:
   /Library/Developer/CommandLineTools/usr/bin/python3

2. Python Version:
   3.9.6 (default, Dec  2 2025, 07:27:58) 
[Clang 17.0.0 (clang-1700.6.3.2)]

3. Virtual Environment:
   ⚠️  NOT DETECTED
   Note: You can still use this notebook, dependencies will be installed automatically.

4. Backtester Package:
   ✅ FOUND (via path): /Users/janussuk/Desktop/OptionsBacktester2/notebooks/../code

5. Dependencies (auto-installing if missing):
   ✅ pandas
   ✅ numpy
   ✅ scipy
   ✅ plotly
   ✅ doltpy

6. Database Path:
   ✅ Found: /Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options

✅ ENVIRONMENT READY - All checks passed!
You may proceed with running the notebook cells.


In [2]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add backtester to path
sys.path.insert(0, str(Path('..') / 'code'))

from datetime import datetime, timedelta
import pandas as pd
import numpy as np

# Backtester framework imports
from backtester.data.dolt_adapter import DoltAdapter
from backtester.engine.backtest_engine import BacktestEngine
from backtester.engine.execution import ExecutionModel
from backtester.engine.data_stream import DataStream
from backtester.strategies.strategy import Strategy
from backtester.structures.straddle import ShortStraddle
from backtester.core.option_structure import OptionStructure

# Analytics imports
from backtester.analytics.metrics import PerformanceMetrics
from backtester.analytics.risk import RiskAnalytics
from backtester.analytics.visualization import Visualization

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

print('✓ All imports successful!')
print(f'Notebook created: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

✓ All imports successful!
Notebook created: 2025-12-17 11:28:14


In [3]:
# Database configuration
DB_PATH = '/Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options'
UNDERLYING = 'SPY'
START_DATE = datetime(2023, 1, 4)
END_DATE = datetime(2025, 1, 29)
INITIAL_CAPITAL = 100000.0

# Connect to database
print(f'Connecting to Dolt database at: {DB_PATH}')
adapter = DoltAdapter(DB_PATH)
adapter.connect()

# Verify database
tables = adapter.get_tables()
print(f'✓ Database connected!')
print(f'Available tables: {tables}')

# Check data availability
date_range = adapter.get_date_range(UNDERLYING)
print(f'Data range for {UNDERLYING}: {date_range[0]} to {date_range[1]}')

Connecting to Dolt database at: /Users/janussuk/Desktop/OptionsBacktester2/dolt_data/options
✓ Database connected!
Available tables: ['option_chain', 'volatility_history']
Data range for SPY: 2019-02-09 00:00:00 to 2025-12-12 00:00:00


In [6]:
# Preview the option_chain and volatility_history tables (SPY only)
print("=" * 80)
print("OPTION_CHAIN TABLE - SPY (first 10 rows)")
print("=" * 80)
option_chain_df = adapter.query_custom("SELECT * FROM option_chain WHERE act_symbol = 'SPY' ORDER BY `date` ASC, `act_symbol` ASC, `expiration` ASC, `strike` ASC, `call_put` ASC LIMIT 100")
display(option_chain_df)

print("\n")
print("=" * 80)
print("VOLATILITY_HISTORY TABLE - SPY (first 10 rows)")
print("=" * 80)
volatility_df = adapter.query_custom("SELECT * FROM volatility_history WHERE act_symbol = 'SPY' LIMIT 10")
display(volatility_df)

OPTION_CHAIN TABLE - SPY (first 10 rows)


,act_symbol,ask,bid,call_put,date,delta,expiration,gamma,rho,strike,theta,vega,vol
0,SPY,40.80,40.37,Call,2019-02-09,0.9908,2019-02-22,0.0013,0.0872,230.00,-0.0266,0.0127,0.3551
1,SPY,0.03,0.02,Put,2019-02-09,-0.0046,2019-02-22,0.0008,-0.0005,230.00,-0.0081,0.0071,0.3223
2,SPY,27.83,27.43,Call,2019-02-09,0.9823,2019-02-22,0.0031,0.0913,243.00,-0.0327,0.0228,0.2633
3,SPY,0.07,0.06,Put,2019-02-09,-0.0137,2019-02-22,0.0026,-0.0014,243.00,-0.0165,0.0185,0.2515
4,SPY,19.46,19.05,Call,2019-02-09,0.9570,2019-02-22,0.0078,0.0919,251.50,-0.0493,0.0481,0.2197
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SPY,1.23,1.22,Put,2019-02-16,-0.1748,2019-03-15,0.0202,-0.0381,266.00,-0.0595,0.1977,0.1658
96,SPY,7.86,7.75,Call,2019-02-16,0.6821,2019-03-15,0.0300,0.1390,272.00,-0.0705,0.2726,0.1539
97,SPY,2.24,2.22,Put,2019-02-16,-0.3061,2019-03-15,0.0314,-0.0668,272.00,-0.0715,0.2691,0.1451
98,SPY,4.24,4.16,Call,2019-02-16,0.5197,2019-03-15,0.0393,0.1072,277.00,-0.0686,0.3052,0.1318




VOLATILITY_HISTORY TABLE - SPY (first 10 rows)


,act_symbol,date,hv_current,hv_month_ago,hv_week_ago,hv_year_high,hv_year_high_date,hv_year_low,hv_year_low_date,iv_current,iv_month_ago,iv_week_ago,iv_year_high,iv_year_high_date,iv_year_low,iv_year_low_date
0,SPY,2019-02-09,0.1589,0.2907,0.2515,0.2923,2019-01-04,0.0590,2018-10-03,0.1288,0.1765,0.1335,0.3157,2018-12-24,0.0806,2018-08-07
1,SPY,2019-02-16,0.1394,0.2802,0.1589,0.2923,2019-01-04,0.0590,2018-10-03,0.1276,0.1554,0.1288,0.3157,2018-12-24,0.0806,2018-08-07
2,SPY,2019-02-23,0.1091,0.2674,0.1394,0.2923,2019-01-04,0.0590,2018-10-03,0.1137,0.1678,0.1276,0.3157,2018-12-24,0.0806,2018-08-07
3,SPY,2019-03-02,0.1048,0.2515,0.1091,0.2923,2019-01-04,0.0590,2018-10-03,0.1052,0.1509,0.1137,0.3157,2018-12-24,0.0806,2018-08-07
4,SPY,2019-03-09,0.0972,0.1589,0.1048,0.2923,2019-01-04,0.0590,2018-10-03,0.1334,0.1293,0.1052,0.3157,2018-12-24,0.0806,2018-08-07
5,SPY,2019-03-16,0.0883,0.1394,0.0972,0.2923,2019-01-04,0.0590,2018-10-03,0.1047,0.1402,0.1334,0.3157,2018-12-24,0.0806,2018-08-07
6,SPY,2019-03-23,0.1056,0.1091,0.0883,0.2923,2019-01-04,0.0590,2018-10-03,0.1448,0.1211,0.1047,0.3157,2018-12-24,0.0806,2018-08-07
7,SPY,2019-03-30,0.1041,0.1042,0.1056,0.2923,2019-01-04,0.0590,2018-10-03,0.1184,0.1195,0.1448,0.3157,2018-12-24,0.0806,2018-08-07
8,SPY,2019-04-06,0.1036,0.0999,0.1041,0.2923,2019-01-04,0.0590,2018-10-03,0.1029,0.1282,0.1184,0.3157,2018-12-24,0.0806,2018-08-07
9,SPY,2019-04-13,0.1052,0.0987,0.1036,0.2923,2019-01-04,0.0590,2018-10-03,0.0971,0.1055,0.1029,0.3157,2018-12-24,0.0806,2018-08-07


In [ ]:
class AdvancedShortStraddleStrategy(Strategy):
    """
    Advanced Short Straddle Strategy with IV-based entry and dynamic exits.
    
    Entry Rules:
    - IV rank > 50 (high implied volatility)
    - DTE between 30-45 days
    - No more than 3 concurrent positions
    
    Exit Rules:
    - 50% profit target
    - 200% stop loss (2x max profit)
    - Exit at 7 DTE or earlier
    """
    
    def __init__(self, name='AdvStraddle', initial_capital=100000.0):
        super().__init__(
            name=name,
            initial_capital=initial_capital,
            position_limits={
                'max_positions': 3,
                'max_total_delta': 50.0,
                'max_total_vega': 300.0,
                'max_capital_utilization': 0.70
            }
        )
        
        # Strategy parameters
        self.iv_rank_threshold = 50.0  # Enter when IV rank > 50
        self.min_entry_dte = 30
        self.max_entry_dte = 45
        self.exit_dte = 7  # Exit at or before 7 DTE
        self.profit_target_pct = 0.50  # 50% of max profit
        self.stop_loss_pct = 2.00  # 200% of max profit (loss)
        self.contracts_per_trade = 1
        
        # Track IV history for rank calculation
        self.iv_history = []
        self.iv_lookback = 90  # days for IV rank calculation
        
    def calculate_iv_rank(self, current_iv):
        """Calculate IV rank (percentile over lookback period)."""
        if len(self.iv_history) < 10:
            return 50.0  # Default to mid-range if insufficient data
        
        iv_values = list(self.iv_history) + [current_iv]
        percentile = (np.searchsorted(sorted(iv_values), current_iv) / len(iv_values)) * 100
        return percentile
    
    def should_enter(self, market_data):
        """Determine if we should enter a new position."""
        # Update IV history
        current_iv = market_data.get('iv', 0.20)
        self.iv_history.append(current_iv)
        if len(self.iv_history) > self.iv_lookback:
            self.iv_history.pop(0)
        
        # Calculate IV rank
        iv_rank = self.calculate_iv_rank(current_iv)
        
        # Entry conditions
        high_iv = iv_rank > self.iv_rank_threshold
        max_positions_ok = self.num_open_positions < self._position_limits['max_positions']
        
        return high_iv and max_positions_ok
    
    def should_exit(self, structure, market_data):
        """Determine if we should exit a position."""
        # Calculate days to expiration
        current_date = market_data.get('date', datetime.now())
        if hasattr(structure, 'call_option'):
            expiration = structure.call_option.expiration
            dte = (expiration - current_date).days
        else:
            dte = 999  # Default high value if expiration not found
        
        # Calculate P&L percentage
        try:
            current_pnl = structure.calculate_pnl()
            max_profit = structure.max_profit if hasattr(structure, 'max_profit') else abs(structure.net_premium)
            
            if max_profit > 0:
                pnl_pct = current_pnl / max_profit
            else:
                pnl_pct = 0.0
        except:
            pnl_pct = 0.0
        
        # Exit conditions
        profit_target_hit = pnl_pct >= self.profit_target_pct
        stop_loss_hit = pnl_pct <= -self.stop_loss_pct
        dte_exit = dte <= self.exit_dte
        
        return profit_target_hit or stop_loss_hit or dte_exit
    
    def create_structure(self, market_data):
        """
        Create a short straddle structure from market data.
        
        Selects ATM strike with DTE in the target range.
        """
        option_chain = market_data.get('option_chain')
        if option_chain is None or option_chain.empty:
            return None
        
        current_date = market_data.get('date', datetime.now())
        spot = market_data.get('spot', 0.0)
        
        # Calculate DTE for each option
        option_chain = option_chain.copy()
        option_chain['dte'] = (pd.to_datetime(option_chain['expiration']) - current_date).dt.days
        
        # Filter by target DTE range
        target_dte_options = option_chain[
            (option_chain['dte'] >= self.min_entry_dte) & 
            (option_chain['dte'] <= self.max_entry_dte)
        ]
        
        if target_dte_options.empty:
            return None
        
        # Find ATM strike (closest to spot)
        target_dte_options['strike_diff'] = abs(target_dte_options['strike'] - spot)
        atm_strike = target_dte_options.loc[target_dte_options['strike_diff'].idxmin(), 'strike']
        
        # Get target expiration (closest to 37 DTE, middle of range)
        target_dte = 37
        target_dte_options['dte_diff'] = abs(target_dte_options['dte'] - target_dte)
        target_expiration = target_dte_options.loc[target_dte_options['dte_diff'].idxmin(), 'expiration']
        
        # Filter for ATM strike and target expiration
        atm_options = option_chain[
            (option_chain['strike'] == atm_strike) &
            (option_chain['expiration'] == target_expiration)
        ]
        
        # Get call and put
        calls = atm_options[atm_options['call_put'] == 'Call']
        puts = atm_options[atm_options['call_put'] == 'Put']
        
        if calls.empty or puts.empty:
            return None
        
        call_data = calls.iloc[0]
        put_data = puts.iloc[0]
        
        # Use mid price (average of bid and ask)
        call_price = (call_data['bid'] + call_data['ask']) / 2.0 if call_data['ask'] > 0 else call_data['bid']
        put_price = (put_data['bid'] + put_data['ask']) / 2.0 if put_data['ask'] > 0 else put_data['bid']
        
        # Create short straddle
        try:
            straddle = ShortStraddle.create(
                underlying=UNDERLYING,
                strike=atm_strike,
                expiration=pd.to_datetime(target_expiration).to_pydatetime(),
                call_price=call_price,
                put_price=put_price,
                quantity=self.contracts_per_trade,
                entry_date=current_date,
                underlying_price=spot,
                call_iv=call_data.get('vol', 0.20),
                put_iv=put_data.get('vol', 0.20)
            )
            return straddle
        except Exception as e:
            print(f'Failed to create straddle: {e}')
            return None

# Create strategy instance
strategy = AdvancedShortStraddleStrategy(
    name='AdvancedShortStraddle',
    initial_capital=INITIAL_CAPITAL
)

print(f'✓ Strategy created: {strategy.name}')
print(f'Parameters:')
print(f'  - IV Rank Threshold: {strategy.iv_rank_threshold}%')
print(f'  - Entry DTE Range: {strategy.min_entry_dte}-{strategy.max_entry_dte} days')
print(f'  - Exit DTE: {strategy.exit_dte} days')
print(f'  - Profit Target: {strategy.profit_target_pct:.0%}')
print(f'  - Stop Loss: {strategy.stop_loss_pct:.0%}')
print(f'  - Max Positions: {strategy._position_limits["max_positions"]}')